In [16]:
import minsearch
import json
from openai import OpenAI
client = OpenAI()

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dic in docs_raw:
    for doc in course_dic['documents']:
        doc['course'] = course_dic['course']
        documents.append(doc)


In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
boost = {'question':3.0, 'section':0.5}

results = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course':'data-engineering-zoomcamp'},
    num_results=5
)

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [11]:
def search(query):
    boost = {'question':3.0, 'section':0.5}

    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict={'course':'data-engineering-zoomcamp'},
        num_results=5
    )
    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assitstant. Anwser the QUESTION base on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION

    If the CONTEXT doesn't contain the answer, output NONE 

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role":"user", "content":prompt}]
    )

    return response.choices[0].message.content

In [14]:
def rag(query):
    search_results= search(query)
    prompt=build_prompt(query, search_results)
    answer=llm(prompt)
    return answer

In [17]:
#query = 'the corse already started. can I still enroll?'
query = 'how do I run kafka?'
rag(query)


'To run the Java Kafka producer, you can execute the following command in the project directory:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor running the Python Kafka producer, make sure you create a virtual environment and install the necessary packages as per the provided instructions.'

In [18]:
rag('the corse already started. can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."